In [3]:
from pyarrow import fs, hdfs
hdfs.HadoopFileSystem("hdfs://namenode:9000")
conexao = fs.HadoopFileSystem(host="namenode", port=9000)

/tmp/ipykernel_106/23586936.py:2: FutureWarning: pyarrow.hdfs.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  hdfs.HadoopFileSystem("hdfs://namenode:9000")


In [39]:
#navegar pelas pastas
conexao.get_file_info(fs.FileSelector('/', recursive=True))

[<FileInfo for '/pasta_avo1': type=FileType.Directory>,
 <FileInfo for '/pasta_avo2': type=FileType.Directory>,
 <FileInfo for '/pasta_avo2/pasta_pai': type=FileType.Directory>]

In [5]:
#Criar pastas
conexao.create_dir('/pasta_avo1')

In [7]:
#Criar pastas e subpastas
conexao.create_dir('/pasta_avo2/pasta_pai/pasta_filho')

In [8]:
#Remover pastas
conexao.delete_dir('/pasta_avo2/pasta_pai/pasta_filho')

In [27]:
#Criar Arquivos
caminho_arquivo_origem = '/home/jovyan/work/Hotel_Reviews.csv'
caminho_arquivo_destino = '/pasta_avo1/hotel.csv'

#conexao.copy_file(caminho_arquivo_origem, caminho_arquivo_destino)

with conexao.open_output_stream(caminho_arquivo_destino) as stream:
    stream.write(open(caminho_arquivo_origem, 'rb').read())

In [28]:
#Ler um conteudo de um arquivo
with conexao.open_input_stream(caminho_arquivo_destino) as f:
    print(f.read(100))

b'Review,Rating\n"nice hotel expensive parking got good deal stay hotel anniversary, arrived late eveni'


In [30]:
#Mover arquivos de uma pasta para outra
caminho_arquivo_destino_movimentacao = '/arquivos_spark/atividade/arquivo_hotel.csv'

conexao.create_dir('/arquivos_spark/atividade')
conexao.move(caminho_arquivo_destino, caminho_arquivo_destino_movimentacao)

In [32]:
#copiar arquivo
conexao.copy_file(caminho_arquivo_destino_movimentacao, '/arquivos_spark/atividade/arquivo_hotel2.csv')

In [34]:
#Remover arquivos
conexao.delete_file('/arquivos_spark/atividade/arquivo_hotel2.csv')

In [36]:
#limpar pasta
conexao.delete_dir_contents('/arquivos_spark/atividade/')

In [38]:
#excluir a pasta junto com arquivos
conexao.delete_dir('/arquivos_spark')

In [ ]:
#NOVO CODIGO PARA LER O ARQUIVO DE FORMA FRACIONADA

# Tamanho do bloco (em bytes)
# Como nao e possivel enviar o arquivo inteiro por meio da celula anteriormente aprendida na aula. Essa ira substituir!
# Esse script quebra o arquivo em blocos. 
#Ref: Fornecido pela monitoria. (Esta funcional)

#Arquivo: training.1600000.processed.noemoticon.csv

tamanho_bloco = 1024 * 1024  # 1 MB

# Abrir o arquivo de origem para leitura
with open(caminho_arquivo_origem, 'rb') as arquivo_origem:
    # Abrir o arquivo de destino para escrita no HDFS
    with conexao.open_output_stream(caminho_arquivo_destino) as stream_destino:
        while True:
            # Ler um bloco do arquivo de origem
            bloco = arquivo_origem.read(tamanho_bloco)
            # Se o bloco estiver vazio, terminar a leitura
            if not bloco:
                break
            # Escrever o bloco no arquivo de destino no HDFS
            stream_destino.write(bloco)